## GPU

In [1]:
!nvidia-smi

Tue Dec 21 20:38:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:1A:00.0 Off |                  N/A |
| 30%   31C    P8    24W / 350W |    257MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:1B:00.0 Off |                  N/A |
| 30%   

### 1.计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')     # 获取计算设备对象，用于指定计算设备

(device(type='cpu'),
 <torch.cuda.device at 0x7f961c389fa0>)

查看可用GPU数量

In [3]:
torch.cuda.device_count()

10

这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [4]:
def try_gpu(i=0):
    '''如果存在，则返回GPU(i)，否则返回CPU()'''
    if torch.cuda.device_count() > i:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    '''返回所有可用的GPU，如果没有GPU，则返回[cpu(), ]'''
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(0), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0),
  device(type='cuda', index=1),
  device(type='cuda', index=2),
  device(type='cuda', index=3),
  device(type='cuda', index=4),
  device(type='cuda', index=5),
  device(type='cuda', index=6),
  device(type='cuda', index=7),
  device(type='cuda', index=8),
  device(type='cuda', index=9)])

### 2.张量与GPU

查询张量所在的设备

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作（比如两个张量求和），它们都必须在同一个设备上。

#### 2.1在GPU上创建张量

In [6]:
X = torch.ones(2, 3, device=try_gpu())
Y = torch.rand(2, 3, device=try_gpu(1))
print(X)
print(Y)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[0.2164, 0.9456, 0.9830],
        [0.4012, 0.2583, 0.7594]], device='cuda:1')


#### 2.2复制张量到GPU

In [7]:
X + Y

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1!

In [8]:
Z = X.cuda(1)
print(Z)
print(X)
print(Z + Y)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1.2164, 1.9456, 1.9830],
        [1.4012, 1.2583, 1.7594]], device='cuda:1')


In [10]:
Z.cuda(1) is Z      # 复制到本设备，不会创建新的

True

#### 2.3旁注

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。但是**在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多**。这也使得并行化变得更加困难，因为我们必须等待数据被发送（或者接收），然后才能继续进行更多的操作。这就是为什么拷贝操作要格外小心。
- 根据经验，多个小操作比一个大操作糟糕得多。
- 此外，一次执行几个操作比代码中散布的许多单个操作要好得多（除非你确信自己在做什么）。
- 如果一个设备必须等待另一个设备才能执行其他操作，那么这样的操作可能会阻塞。（有流水化技术？）

最后，当我们**打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会首先将其复制到内存中**，这会导致额外的传输开销。更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成。

### 3.神经网络与GPU

调用net.to()方法，可以将网络参数挪动到某个计算设备上

In [13]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)      # 同样地，net(Y)不能运算

tensor([[-0.5087],
        [-0.5087]], device='cuda:0', grad_fn=<AddmmBackward0>)